In [ ]:
import numpy as np
import pandas as pd
from fourinarowfunctions import *

%load_ext autoreload
%autoreload 2

Provide the directory where the data is kept

In [ ]:
direc = '../data/'

Read parameters and calculate feature drop rate and heuristic quality

In [ ]:
df = pd.read_csv(direc + 'paramsMatrix.csv')
params = np.array([expand_params(r) for r in df.values[:,-10:].astype(float)])

df['feature drop rate'] = params[:,-1]
df['heuristic quality'] = np.apply_along_axis(get_heuristic_quality,1,params)

We compute planning depth on the cluster. First, export the parameters

In [ ]:
np.savetxt(direc + 'params.txt', params)

Create a directory on the cluster and move params.txt into it. Edit the compute_planning_depth.sh shell script so that the directory is correct, and change the number of jobs to the number of lines in params.txt (in this case 370). Run the script, then zip the depth folder, move it back to your local computer and unzip in the directory where paramsMatrix.csv is stored. 

In [ ]:
depth = np.vstack([np.loadtxt(direc + f"depth/depth_fourinarow-dev_{i // 5}_{i % 5 + 1}.txt") for i in range(len(params))])
df['depth'] = depth

In [ ]:
df

In [ ]:
df.to_csv(direc + 'params_with_metrics.csv')

Calculate Elo

In [ ]:
results_df = pd.read_csv(direc + 'forElo.csv')
results = [[row['subject'].split(':')[1],str(row['level']),row['outcome']] if row['user_color']=='black' else
           [str(row['level']),row['subject'].split(':')[1],-row['outcome']] for _,row in results_df.iterrows()]
create_bayeselo_input(results, direc + 'games.pgn')

In [ ]:
bayeselo_direc = './'

elo = run_bayeselo(bayeselo_direc,[direc + 'tournament_results_short.pgn', direc + 'games.pgn'])
elo = {key : val for key,val in elo.items() if key.startswith('R_')}

In [ ]:
elo